### Create Block

Blocks are the basic storage unit. Every **view** will reference some **block** where the data is stored.

Before starting we need to import the **pyJvsip** module.  I use **pjv** for the alias.

In [1]:
import pyJvsip as pjv

You can create a simple block with the **Block** class.  Blocks are all typed and have length. The length and type are imutable. Once it is created that is what you have.  For more information on VSIPL block types see the VSIPL specification.  Note the blocks contain VSIPL defined scalar values and the length is the number of scalars the block can store. For now know we have the following types.

* **'block_f'** Float 
* **'block_d'** Double 
* **'cblock_f'** Complex Float 
* **'cblock_d'** Complex Double
* **'block_vi'** Vector index
* **'block_mi'** Matrix index
* **'block_bl'** Boolean
* **'block_si'** Short integer
* **'block_i'** Integer
* **'block_uc'** Unsigned Char

Below we create a block of type double to store up to 500 values.

In [2]:
ablock = pjv.Block('block_d',500)

### Create View

You cannot do much with just a block.  You generally work with views of type **matrix** or **vector**.

Blocks have a **bind** method which will create the view and bind it to the block.

The bind method takes an offset, stride, and length for vectors and offset, row stride, row length, column stride, and column length for matrices.  See C VSIPL specification for information on how data in blocks is referenced by the view.  The view will get it's type from the block and call sequence. Note that view data cannot exceed the data length of the block.

View types are as follows

**vector**
* **'vview_f'**
* **'vview_d'**
* **'cvview_f'**
* **'cvview_d'**
* **'vview_si'**
* **'vview_i'**
* **'vview_uc'**
* **'vview_mi'**
* **'vview_vi'**
* **'vview_bl'**

**matrix**
* **'mview_f'**
* **'mview_d'**
* **'cmview_f'**
* **'cmview_d'**
* **'mview_si'**
* **'mview_i'**
* **'mview_uc'**
* **'mview_bl'**

Note that scalar types matrix and vector index (**mi** and **vi**) are not supported by matrix views.

Create vector of length 10

In [3]:
stride = 1
length = 10
offset = 0
avector = ablock.bind(offset,stride,length)
avector.type

'vview_d'

create matrix of size (4,5)

In [4]:
rowstride = 1
colstride = 5
rowlength = 5
collength = 4
amatrix = ablock.bind(offset,colstride,collength,rowstride,rowlength)
amatrix.type

'mview_d'

Blocks have a convenience method **vector** which returns a unit stride zero offset vector of block length. There is also a generator function called ramp. We fill ablock with data 0.0, 1.0,..., ablock.length-1

In [5]:
ablock.vector.ramp(0,1)

In the above operation we wanted to fill the block with data using ramp.  We created the vector for ramp to use but we did not set it egual to anything so the view was not kept and was deleted by python after ramp was done with it. 

Now print *avector*

In [6]:
avector.mprint('%0.1f')

[ 0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0]



Now print *amatrix*

In [7]:
amatrix.mprint('%0.1f')

[ 0.0  1.0  2.0  3.0  4.0;
  5.0  6.0  7.0  8.0  9.0;
  10.0  11.0  12.0  13.0  14.0;
  15.0  16.0  17.0  18.0  19.0]



### Convenience methods 

Frequently you want a view of the same size and type as another view.  There are three methods for doing this called **clone**, **copy**, and **empty**.  

#### **clone**
The clone method will create a new view on the same block as the calling view and the cloned view will exactly reference the same data as the calling view.  Cloning is handy if you want a new view to modify the attributes but don't want to change the attributes on the calling view.

#### **copy**
The copy method will create a new view of the same size as the calling method and a new block to store the data.  The new block will (probably) not be the same size as the calling views block.  It will be the exact size to fit the views data. The calling views data is copied into the new view.

#### **empty**
The empty method will do the same thing as the copy method except it will not copy the data.  Frequently you need another view of the same size and type but the data of the calling view is not important.

Below we have some examples.

In [8]:
amatrixClone = amatrix.clone

In [9]:
amatrixClone[1,1] = 100
amatrix.mprint('%0.1f')

[ 0.0  1.0  2.0  3.0  4.0;
  5.0  100.0  7.0  8.0  9.0;
  10.0  11.0  12.0  13.0  14.0;
  15.0  16.0  17.0  18.0  19.0]



In [10]:
amatrixCopy = amatrix.copy
amatrixCopy[1,1] = 6.0
amatrixCopy.mprint('%0.1f')
amatrix.mprint('%0.1f')

[ 0.0  1.0  2.0  3.0  4.0;
  5.0  6.0  7.0  8.0  9.0;
  10.0  11.0  12.0  13.0  14.0;
  15.0  16.0  17.0  18.0  19.0]

[ 0.0  1.0  2.0  3.0  4.0;
  5.0  100.0  7.0  8.0  9.0;
  10.0  11.0  12.0  13.0  14.0;
  15.0  16.0  17.0  18.0  19.0]



In [11]:
amatrixEmpty = amatrix.empty
amatrixEmpty.fill(0.0)
amatrixEmpty.mprint('%0.1f')

[ 0.0  0.0  0.0  0.0  0.0;
  0.0  0.0  0.0  0.0  0.0;
  0.0  0.0  0.0  0.0  0.0;
  0.0  0.0  0.0  0.0  0.0]



In [12]:
print(amatrix.block.length)
print(amatrixClone.block.length)
print(amatrixEmpty.block.length)
print(amatrixCopy.block.length)

500
500
20
20


#### Note
**amatrix.block** and **amatrixClone.block** are the same block.  **amatrixEmpty.block** and **amatrixCopy.block** are new blocks and not related to each other.